# 🚀 **Passo 4: Avaliando a Qualidade das Respostas**

## **Aula 4.1: Por Que Qualidade é Importante?**

---

### **Tá, mas o que é avaliação de qualidade?**

Imagina que você pediu um hambúrguer num restaurante. O garçom trouxe em 2 minutos (rápido!), mas quando você mordeu, tava cru por dentro! Velocidade sem qualidade não serve pra nada. É exatamente isso que vamos avaliar aqui - não só se o modelo é rápido, mas se ele faz um trabalho BOM! 😄

**Por que avaliação de qualidade é importante?**

Um modelo pode ser super rápido e barato, mas se ele gera resumos que não fazem sentido ou perdem informações importantes, de que adianta? É como ter um funcionário que trabalha rápido mas faz tudo errado - não é útil pra ninguém!

### **Entendendo a Avaliação LLM-as-a-Judge**

LLM-as-a-Judge funciona assim:

1. **Pega** as respostas do seu modelo junto com respostas de referência (verdade absoluta)
2. **Usa** um modelo avaliador especializado pra julgar a qualidade
3. **Fornece** scores padronizados em dimensões como correção, completude e estilo

Essa abordagem oferece várias vantagens sobre avaliação humana tradicional:
- **Consistência**: Aplica os mesmos padrões em todas as respostas
- **Escalabilidade**: Pode avaliar milhares de respostas rapidamente
- **Objetividade**: Reduz viés humano no processo
- **Reprodutibilidade**: Produz resultados consistentes pra mesmos inputs

### **Nossa Abordagem de Avaliação**

Pra esse workshop, vamos:
1. **Formatar** as respostas geradas no Passo 3 pra avaliação
2. **Criar** jobs de avaliação separados pra cada modelo
3. **Fazer upload** dos datasets pro S3 pra processamento
4. **Configurar** e iniciar os jobs de avaliação
5. **Armazenar** referências dos jobs pra análise no próximo passo

Vamos começar garantindo que temos as dependências necessárias instaladas:

---

**🖼️ Sugestão de imagem**: Um juiz com martelo avaliando respostas de IA

In [ ]:
# 🛠️ IMPORTANDO AS FERRAMENTAS NECESSÁRIAS
import boto3
import json
import random
from datetime import datetime
from typing import List, Dict, Any, Optional
import pandas as pd
import glob
import os
from IPython.display import display

# Inicializando clientes AWS
bedrock_client = boto3.client('bedrock')
s3_client = boto3.client('s3')

print("✅ Ferramentas importadas! Vamos avaliar a qualidade!")

### **Pré-requisitos**

Pra esse notebook funcionar com sucesso, você precisa de:

- **Role IAM com Permissões Adequadas**: Pra criar jobs de avaliação e acessar S3
  - _Nota: No workshop hospedado, essas roles são pré-configuradas pra você_
  - _Pra aprendizes autodidatas: Siga as instruções do workshop pra criar a role necessária_
- **Bucket S3**: Pra armazenar datasets de avaliação e resultados
  - _Nota: O workshop cria isso automaticamente com as permissões corretas_
  - _Pra aprendizes autodidatas: Crie seu próprio bucket e atualize o nome do bucket neste notebook_
- **Modelos de Avaliação**: Acesso a modelos como Claude Haiku/Sonnet, Amazon Nova

In [ ]:
# ⚙️ CONFIGURAÇÃO AWS
account_id = boto3.client('sts').get_caller_identity().get('Account')

ROLE_ARN = f"arn:aws:iam::{account_id}:role/service-role/Bedrock-LLM-as-a-Judge-ExecutionRole"

BUCKET_NAME = f"genai-evaluation-migration-bucket-{account_id}"
PREFIX = "genai_migration"

print(f"🏢 Account ID: {account_id}")
print(f"🔑 Role ARN: {ROLE_ARN}")
print(f"🪣 Bucket: {BUCKET_NAME}")
print(f"📁 Prefix: {PREFIX}")

### **Carregando Nosso Progresso**

Vamos carregar nosso dataframe de tracking do Passo 3, que contém informações sobre nosso modelo fonte e os modelos candidatos que vamos avaliar. Esse dataframe vai servir como nosso repositório central pra todas as métricas de avaliação durante o workshop.

In [ ]:
# 📊 CARREGANDO NOSSO TRACKING
evaluation_tracking_file = '../data/evaluation_tracking.csv'
evaluation_tracking = pd.read_csv(evaluation_tracking_file)
display(evaluation_tracking)

print("\n💡 Perfeito! Agora temos nosso plano de avaliação carregado.")

### **Preparação de Dados pra Avaliação de Qualidade**

#### **Formatando Respostas pro LLM-as-a-Judge**

O LLM-as-a-Judge precisa dos dados num formato específico. É como preparar a comida pro juiz de um concurso culinário - tem que estar na apresentação certa!

Vamos formatar as respostas que geramos no Passo 3 pro formato que o LLM-as-a-Judge espera:

In [ ]:
# 📝 FUNÇÃO PRA FORMATAR DADOS PRA AVALIAÇÃO
def format_data_for_evaluation(model_id, output_file_path):
    """
    Formata os dados de um modelo específico pro formato do LLM-as-a-Judge.
    É como traduzir um documento pra uma língua que o juiz entende!
    """
    
    try:
        # Carregando os dados do modelo
        model_data = pd.read_csv(output_file_path)
        
        # Filtrando apenas resultados bem-sucedidos
        successful_data = model_data[model_data['status'] == 'success']
        
        if len(successful_data) == 0:
            print(f"⚠️ Nenhum resultado bem-sucedido encontrado para {model_id}")
            return None
        
        # Formatando pro formato do LLM-as-a-Judge
        evaluation_data = []
        
        for _, row in successful_data.iterrows():
            evaluation_entry = {
                'prompt': row['document'],  # O documento original
                'referenceResponse': row['referenceResponse'],  # Resposta de referência
                'modelResponses': [
                    {
                        'response': row['model_response'],  # Resposta do modelo
                        'modelId': model_id
                    }
                ]
            }
            evaluation_data.append(evaluation_entry)
        
        print(f"✅ {model_id}: {len(evaluation_data)} amostras formatadas")
        return evaluation_data
        
    except Exception as e:
        print(f"❌ Erro ao formatar dados para {model_id}: {str(e)}")
        return None

print("✅ Função de formatação criada! Vamos preparar os dados.")

In [ ]:
# �� PREPARANDO DADOS PRA TODOS OS MODELOS
print("�� PREPARANDO DADOS PRA AVALIAÇÃO...")
print("=" * 50)

# Encontrando todos os arquivos de saída
output_directory = '../outputs'
csv_files = glob.glob(os.path.join(output_directory, 'document_summarization_*.csv'))

evaluation_datasets = {}

for file_path in csv_files:
    # Extraindo o nome do modelo do nome do arquivo
    filename = os.path.basename(file_path)
    
    # Identificando o modelo baseado no nome do arquivo
    if 'source_model' in filename:
        model_id = 'source_model'
    elif 'amazon.nova-lite' in filename:
        model_id = 'amazon.nova-lite-v1:0'
    elif 'claude-3-5-haiku' in filename:
        model_id = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
    else:
        print(f"⚠️ Modelo não reconhecido em: {filename}")
        continue
    
    print(f"\n🎯 Processando: {model_id}")
    
    # Formatando os dados
    formatted_data = format_data_for_evaluation(model_id, file_path)
    
    if formatted_data:
        evaluation_datasets[model_id] = formatted_data
        
        # Salvando localmente também (pra backup)
        local_filename = f'../outputs/quality_evaluation.{model_id}.jsonl'
        with open(local_filename, 'w') as f:
            for entry in formatted_data:
                f.write(json.dumps(entry) + '\n')
        
        print(f"�� Dados salvos localmente: {local_filename}")

print(f"\n✅ Total de modelos preparados: {len(evaluation_datasets)}")

### **Upload dos Datasets pro S3**

Agora vamos fazer upload dos datasets formatados pro S3. O LLM-as-a-Judge precisa dos dados no S3 pra processar. É como enviar os documentos pro tribunal!

In [ ]:
# ☁️ FUNÇÃO PRA FAZER UPLOAD PRO S3
def upload_dataset_to_s3(model_id, evaluation_data):
    """
    Faz upload do dataset de avaliação pro S3.
    É como enviar os documentos pro tribunal via correio!
    """
    
    try:
        # Criando nome único pro arquivo
        timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
        s3_key = f"{PREFIX}/evaluation_datasets/{model_id.replace(':', '-')}_{timestamp}.jsonl"
        
        # Convertendo dados pra JSONL
        jsonl_content = ''
        for entry in evaluation_data:
            jsonl_content += json.dumps(entry) + '\n'
        
        # Fazendo upload pro S3
        s3_client.put_object(
            Bucket=BUCKET_NAME,
            Key=s3_key,
            Body=jsonl_content.encode('utf-8'),
            ContentType='application/json'
        )
        
        print(f"✅ Upload concluído: s3://{BUCKET_NAME}/{s3_key}")
        return s3_key
        
    except Exception as e:
        print(f"❌ Erro no upload para {model_id}: {str(e)}")
        return None

print("✅ Função de upload criada! Vamos enviar os dados pro S3.")

In [ ]:
# 🚀 FAZENDO UPLOAD DE TODOS OS DATASETS
print("☁️ FAZENDO UPLOAD DOS DATASETS...")
print("=" * 50)

s3_keys = {}

for model_id, evaluation_data in evaluation_datasets.items():
    print(f"\n📤 Upload para: {model_id}")
    
    s3_key = upload_dataset_to_s3(model_id, evaluation_data)
    
    if s3_key:
        s3_keys[model_id] = s3_key
        
        # Atualizando o tracking
        evaluation_tracking.loc[evaluation_tracking['model'] == model_id, 'quality_evaluation_output'] = s3_key

print(f"\n✅ Uploads concluídos: {len(s3_keys)} datasets")
print("\n�� Resumo dos uploads:")
for model_id, s3_key in s3_keys.items():
    print(f"• {model_id}: {s3_key}")

### **Configurando e Iniciando Jobs de Avaliação**

Agora vamos configurar e iniciar os jobs de avaliação do LLM-as-a-Judge. É como abrir os processos no tribunal!

In [ ]:
# ⚖️ FUNÇÃO PRA CRIAR JOB DE AVALIAÇÃO
def create_evaluation_job(model_id, s3_key):
    """
    Cria um job de avaliação LLM-as-a-Judge pra um modelo específico.
    É como abrir um processo no tribunal com todas as evidências!
    """
    
    try:
        # Configurando o job de avaliação
        job_name = f"llmaaj-{model_id.replace(':', '-').replace('.', '-')}-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"
        
        # Configuração do job
        evaluation_config = {
            'jobName': job_name,
            'jobDescription': f'Avaliação de qualidade para {model_id}',
            'roleArn': ROLE_ARN,
            'evaluationConfig': {
                'taskType': 'General',
                'datasetConfig': {
                    'datasetType': 'CustomDataset',
                    's3Uri': f's3://{BUCKET_NAME}/{s3_key}'
                },
                'modelConfig': {
                    'modelId': model_id
                },
                'evaluationMetrics': [
                    'Builtin.Correctness',
                    'Builtin.Completeness',
                    'Builtin.ProfessionalStyleAndTone'
                ],
                'evaluatorConfig': {
                    'evaluatorModelId': 'amazon.nova-pro-v1:0'
                }
            },
            'outputConfig': {
                's3Uri': f's3://{BUCKET_NAME}/{PREFIX}/evaluation_results/'
            }
        }
        
        # Criando o job
        response = bedrock_client.create_evaluation_job(**evaluation_config)
        
        job_arn = response['evaluationJobArn']
        
        print(f"✅ Job criado: {job_name}")
        print(f"�� ARN: {job_arn}")
        
        return job_arn
        
    except Exception as e:
        print(f"❌ Erro ao criar job para {model_id}: {str(e)}")
        return None

print("✅ Função de criação de job criada! Vamos iniciar as avaliações.")

In [ ]:
# �� INICIANDO TODOS OS JOBS DE AVALIAÇÃO
print("⚖️ INICIANDO JOBS DE AVALIAÇÃO...")
print("=" * 50)

job_arns = {}

for model_id, s3_key in s3_keys.items():
    print(f"\n🎯 Iniciando avaliação para: {model_id}")
    
    job_arn = create_evaluation_job(model_id, s3_key)
    
    if job_arn:
        job_arns[model_id] = job_arn
        
        # Atualizando o tracking
        evaluation_tracking.loc[evaluation_tracking['model'] == model_id, 'quality_evaluation_jobArn'] = job_arn

print(f"\n✅ Jobs iniciados: {len(job_arns)}")
print("\n�� Resumo dos jobs:")
for model_id, job_arn in job_arns.items():
    print(f"• {model_id}: {job_arn}")

### **Monitorando o Status dos Jobs**

Agora vamos monitorar o status dos jobs de avaliação. É como acompanhar o progresso dos processos no tribunal!

In [ ]:
# �� FUNÇÃO PRA MONITORAR STATUS DOS JOBS
def check_job_status(job_arn):
    """
    Verifica o status de um job de avaliação.
    É como verificar se o processo já foi julgado!
    """
    
    try:
        response = bedrock_client.get_evaluation_job(jobIdentifier=job_arn)
        status = response['status']
        
        return status
        
    except Exception as e:
        print(f"❌ Erro ao verificar status: {str(e)}")
        return 'ERROR'

print("✅ Função de monitoramento criada! Vamos verificar os status.")

In [ ]:
# �� VERIFICANDO STATUS DOS JOBS
print("�� STATUS DOS JOBS DE AVALIAÇÃO:")
print("=" * 50)

for model_id, job_arn in job_arns.items():
    print(f"\n🎯 {model_id}:")
    
    status = check_job_status(job_arn)
    
    if status == 'InProgress':
        print(f"  �� Status: {status} (Em andamento)")
    elif status == 'Completed':
        print(f"  ✅ Status: {status} (Concluído)")
    elif status == 'Failed':
        print(f"  ❌ Status: {status} (Falhou)")
    else:
        print(f"  ⚠️ Status: {status} (Desconhecido)")

print("\n💡 Os jobs podem demorar alguns minutos para completar.")
print("💡 Você pode verificar o status novamente executando esta célula.")

### **Salvando Nosso Progresso**

Vamos salvar nosso dataframe atualizado com todas as informações dos jobs de avaliação. Isso vai ser crucial pro próximo passo!

In [ ]:
# 💾 SALVANDO O TRACKING ATUALIZADO
evaluation_tracking.to_csv('../data/evaluation_tracking.csv', index=False)

print("✅ Tracking atualizado salvo!")
print("\n📊 RESUMO DO QUE FIZEMOS:")
print(f"• Modelos preparados: {len(evaluation_datasets)}")
print(f"• Datasets enviados pro S3: {len(s3_keys)}")
print(f"• Jobs de avaliação criados: {len(job_arns)}")
print(f"• Arquivo salvo: ../data/evaluation_tracking.csv")

# Mostrando o status final
display(evaluation_tracking[['model', 'quality_evaluation_jobArn', 'quality_evaluation_output']])

### **Verificação Final dos Jobs**

Vamos fazer uma verificação final pra ver se todos os jobs estão rodando corretamente:

In [ ]:
# 🔍 VERIFICAÇÃO FINAL DOS JOBS
print("🔍 VERIFICAÇÃO FINAL DOS JOBS:")
print("=" * 40)

completed_jobs = 0
in_progress_jobs = 0
failed_jobs = 0

for model_id, job_arn in job_arns.items():
    status = check_job_status(job_arn)
    
    if status == 'Completed':
        completed_jobs += 1
        print(f"✅ {model_id}: Concluído")
    elif status == 'InProgress':
        in_progress_jobs += 1
        print(f"�� {model_id}: Em andamento")
    elif status == 'Failed':
        failed_jobs += 1
        print(f"❌ {model_id}: Falhou")
    else:
        print(f"⚠️ {model_id}: Status desconhecido ({status})")

print(f"\n�� RESUMO:")
print(f"• Concluídos: {completed_jobs}")
print(f"• Em andamento: {in_progress_jobs}")
print(f"• Falharam: {failed_jobs}")

if in_progress_jobs > 0:
    print(f"\n⏰ {in_progress_jobs} jobs ainda estão rodando.")
    print("�� Você pode aguardar alguns minutos e executar esta célula novamente.")
    print("�� Ou prosseguir para o próximo passo e verificar os resultados depois.")

### **Resumo do Passo 4**

 **Parabéns!** Você acabou de completar o quarto passo da nossa jornada de migração. Vamos recapitular o que fizemos:

✅ **Entendemos a importância da qualidade**: Velocidade sem qualidade não serve
✅ **Formatamos dados pra avaliação**: Preparamos tudo pro LLM-as-a-Judge
✅ **Fizemos upload pro S3**: Enviamos os dados pro "tribunal"
✅ **Criamos jobs de avaliação**: Abrimos os processos
✅ **Monitoramos o progresso**: Acompanhamos o status dos jobs
✅ **Salvamos referências**: Tracking atualizado com ARNs dos jobs

### **O Que Vem no Próximo Passo**

No próximo notebook, vamos fazer a **comparação final**! É como reunir todos os juízes pra dar o veredicto final. Vamos consolidar todas as métricas (latência, qualidade e custo) e gerar um relatório completo que vai nos ajudar a tomar a decisão final sobre qual modelo usar.

---

**💡 Dica do Pedro**: Avaliação de qualidade é crucial! Um modelo pode ser rápido e barato, mas se a qualidade for ruim, você vai ter que refazer tudo depois. É como comprar um carro barato que quebra toda semana!

**🚀 Próximo passo**: Comparação final e geração de relatório